In [ ]:
import pylas
import laspy
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
# Test run with SP3176_P_11321_20171123_20171123.las

In [ ]:
# Use LAStools to convert from Laz to Las
# See: https://community.rockrobotic.com/t/running-laszip-to-compress-your-las-files/154
# Look into lasclip: https://lastools.github.io/download/lasclip_README.txt
laz_dir = "/Users/kevin/cs236g/P_11321_11322"

for laz_file_name in os.listdir(laz_dir):
    
    if laz_file_name.endswith(".laz"):
        
        laz_fname = laz_file_name
        las_fname = laz_file_name[:-4] + ".las"

        !/Users/kevin/CS236G/LAStools-master/bin/laszip -i /Users/kevin/cs236g/P_11321_11322/{laz_fname} -o /Users/kevin/cs236g/LAS/{las_fname}   

In [ ]:
def laz_to_las(input_laz_path, output_las_path):
    
    las = pylas.read(input_laz_path)    
    las = pylas.convert(las)    
    las.write(output_las_path)
    print(f"Save LAS file at: {output_las_path}")
    return output_las_path

In [ ]:
las_no = 'SP3278'
las = laspy.read('/Users/kevin/cs236g/' + las_no + '_P_11321_20171123_20171123.las')
las

In [ ]:
# Convert LAS to numpy array (X=raw integer value x=scaled float value)
lidar_points = np.array((las.x,las.y,las.z,las.intensity,
               las.raw_classification,las.scan_angle_rank)).transpose()
lidar_points

In [ ]:
# Transform to pandas DataFrame
lidar_df = pd.DataFrame(lidar_points)
lidar_df

In [ ]:
# 10000000
lidar_subset = lidar_df.sample(n=1000, random_state=1)
lidar_subset = lidar_subset.reset_index(drop=True)
display(lidar_subset)
print(lidar_subset[2].describe())

In [ ]:
# Transform to geopandas GeoDataFrame
# Original CRS is EPSG:27700
# Transform to EPSG:4326
crs = None
geometry_3D = [shapely.geometry.Point(xyz) for xyz in zip(lidar_subset[0], lidar_subset[1], lidar_subset[2])]
lidar_3D_gdf = gpd.GeoDataFrame(geometry_3D, crs=crs, geometry=geometry_3D)
lidar_3D_gdf.crs = {'init' : 'epsg:27700'} # set correct spatial reference
lidar_3D_gdf = lidar_3D_gdf.drop(columns=[0])
lidar_3D_gdf = lidar_3D_gdf.to_crs(4326)

lidar_3D_gdf['2D_geom'] = [shapely.geometry.Point(xy) for xy in zip(lidar_3D_gdf.geometry.x, lidar_3D_gdf.geometry.y)]
lidar_3D_gdf

In [ ]:
lidar_3D_gdf = lidar_3D_gdf.set_geometry('2D_geom')
print(f"Check name of active geometry column: {lidar_3D_gdf.geometry.name}")
print(f"Check CRS of active geometry column: {lidar_3D_gdf.geometry.crs}")

In [ ]:
buidling_footprints = gpd.read_file("/Users/kevin/cs236g/coventry_building_footprints.geojson")
buidling_footprints = buidling_footprints.loc[buidling_footprints.geometry.apply(lambda x: isinstance(x, shapely.geometry.polygon.Polygon))]
buidling_footprints

In [ ]:
# The spatial join automatically drops the 2D_geom column
points_per_footprint = gpd.sjoin(buidling_footprints, lidar_3D_gdf, how="inner", op='intersects')
points_per_footprint

In [ ]:
points_per_footprint = points_per_footprint.set_geometry('geometry_right')
print(f"Check name of active geometry column: {points_per_footprint.geometry.name}")
print(f"Check CRS of active geometry column: {points_per_footprint.geometry.crs}")

In [ ]:
points_per_footprint['index_id'] = points_per_footprint.index
points_per_footprint

In [ ]:
points_per_footprint_dissolved = points_per_footprint.dissolve('index_id')
points_per_footprint_dissolved

In [ ]:
points_per_footprint_dissolved = points_per_footprint_dissolved.rename_geometry('points_per_footprint')
points_per_footprint_dissolved = points_per_footprint_dissolved.rename(columns={'geometry_left':'footprint_polygon'})
points_per_footprint_dissolved = points_per_footprint_dissolved.reset_index(drop=True)
points_per_footprint_dissolved

In [ ]:
print(f"Check name of active geometry column: {points_per_footprint_dissolved.geometry.name}")
print(f"Check CRS of active geometry column: {points_per_footprint_dissolved.geometry.crs}")

In [ ]:
points_per_footprint_dissolved = points_per_footprint_dissolved.loc[points_per_footprint_dissolved.geometry.apply(lambda x: isinstance(x, shapely.geometry.MultiPoint))]
points_per_footprint_dissolved

In [ ]:
points_per_footprint_dissolved['num_points'] = points_per_footprint_dissolved.apply(lambda row: len(row.points_per_footprint), axis=1)
points_per_footprint_dissolved

In [ ]:
# Convert points_per_footprint back to EPSG:27700
points_per_footprint_dissolved = points_per_footprint_dissolved.to_crs(27700)
points_per_footprint_dissolved = points_per_footprint_dissolved.set_geometry('footprint_polygon')
points_per_footprint_dissolved['footprint_polygon'] = points_per_footprint_dissolved['footprint_polygon'].set_crs('epsg:4326', allow_override=True)
points_per_footprint_dissolved = points_per_footprint_dissolved.to_crs(27700)
points_per_footprint_dissolved

In [ ]:
# Add footprint bounds: minx, miny, maxx, maxy
points_per_footprint_dissolved['footprint_bounds'] = points_per_footprint_dissolved['footprint_polygon'].apply(lambda footprint: footprint.bounds)
# expand footprint polygon bounds into its own dataframe
bounds = points_per_footprint_dissolved['footprint_bounds'].apply(pd.Series)
bounds = bounds.rename(columns = {0:'footprint_minx', 1:'footprint_miny', 2:'footprint_maxx', 3:'footprint_maxy'})
points_per_footprint_dissolved = pd.concat([points_per_footprint_dissolved, bounds], axis=1)
points_per_footprint_dissolved

In [ ]:
# determine the footprint polygon bounds for each building 
# and normalize for each point within the respective object (house) to lie between 0 and 1
copy_gdf = points_per_footprint_dissolved

In [ ]:
def max_z(row):
    max_z = 0
    for elem in list(row.points_per_footprint.geoms):
        if elem.z > max_z:
            max_z = elem.z
    return max_z

def min_z(row):
    min_z = 1000
    for elem in list(row.points_per_footprint.geoms):
        if elem.z < min_z:
            min_z = elem.z
    return min_z

def point_to_numpy(row):
    point_list = []
    for point in list(row.points_per_footprint.geoms):
        point_list.append([point.x, point.y, point.z])
    return point_list

def normalize_points(row):
    normalized_points = []
    footprint_minx = row['footprint_minx']
    footprint_miny = row['footprint_miny']
    footprint_maxx = row['footprint_maxx']
    footprint_maxy = row['footprint_maxy']
    
    for point in list(row.unnormalized_points):
        normalized_x = (point[0] - footprint_minx) / (footprint_maxx - footprint_minx)
        normalized_y = (point[1] - footprint_miny) / (footprint_maxy - footprint_miny)
        normalized_z = (point[2] - row.min_z) / ((row.max_z - row.min_z) + 0.0000001)
        normalized_points.append([normalized_x, normalized_y, normalized_z]) 
    normalized_points_np = np.asarray(normalized_points, dtype=np.float32)
    return normalized_points_np
        
copy_gdf['max_z'] = copy_gdf.apply(max_z, axis=1)
copy_gdf['min_z'] = copy_gdf.apply(min_z, axis=1)
copy_gdf['mean_z'] = copy_gdf.apply(lambda row: (row.max_z + row.min_z)/2, axis=1)
copy_gdf['unnormalized_points'] = copy_gdf.apply(point_to_numpy, axis=1)
copy_gdf['normalized_points'] = copy_gdf.apply(normalize_points, axis=1)
# multipoint to numpy array
copy_gdf

In [ ]:
import pickle
save_path = '/Users/kevin/cs236g/' + las_no + '_PC_Coventry.pickle'
with open(save_path, 'wb') as fp:
    pickle.dump(copy_gdf, fp)